<a href="https://colab.research.google.com/github/kingsgameBen/MLbasic_0223/blob/main/news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

News 新聞分類分析 

檔案先放入

glob.py   大型檔案用的函式庫

glob : 路徑的pattern

*:任意

In [ ]:
from zipfile import ZipFile
f = ZipFile("/content/drive/MyDrive/2021_Python班/chinese_news_trans.zip")
f.extractall()
f = ZipFile("/content/drive/MyDrive/2021_Python班/chinese_news_test.zip")
f.extractall()

In [ ]:
import os
import glob
import pandas as pd
def process(base):
  datas = {"content":[],
          "ans":[]
  }
  # print(os.path.join(base, "*"))
  dirs = glob.glob(os.path.join(base, "*"))
  # print(dirs)
  for d in dirs:
    # print(d)
    fns1 = os.path.join(d, '*.txt')
    fns2 = os.path.join(d, '*.TXT')
    fns = glob.glob(fns1) + glob.glob(fns2)
    for fn in fns:
      with open(fn, "r", encoding="utf-8")as f:
        datas["content"].append(f.read())
      datas["ans"].append(os.path.split(d)[-1])
    df = pd.DataFrame(datas)
  return df
train_df = process("chinese_news_trans")
test_df = process("chinese_news_test")
train_df

1. 正確率
2. 混淆矩陣
3. 輸入input


In [ ]:
u = train_df["ans"].unique()
name2cat = {name:i for i, name in enumerate(u)}
y_train = train_df["ans"].replace(name2cat)
y_test = test_df["ans"].replace(name2cat)
y_train

In [ ]:
# 務必記得載入大字典
from urllib.request import urlretrieve
url = "https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big"
urlretrieve(url, "dict.txt.big")

In [ ]:
import jieba
jieba.set_dictionary("dict.txt.big")
def newscut(p):
  return " ".join(jieba.cut(p))
x_train = train_df["content"].apply(newscut)
x_test = test_df["content"].apply(newscut)
# x_test

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
x_train_count = vec.fit_transform(x_train)
x_test_count = vec.transform(x_test)
print(x_train_count)

In [ ]:
vec.vocabulary_
# vec.vocabulary_["，"]  # 檢查沒有標點符號
x_train_count

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha=0.1)
clf.fit(x_train_count, y_train)

accuracy_score =>0.6 0.9都是對,難以比較

=> 找差值

正確 
100% cat - 50%  0  \
0%   dog - 50%  100 \
$$
(求差值)損失找min \\
Loss \rightarrow (0.5-1)^2 + (0.5-0)^2
$$
\
均方誤差(Mean square error，MSE) \
平均絕對值誤差(Mean absolute error，MAE)\
[ https://chih-sheng-huang821.medium.com/%E6%A9%9F%E5%99%A8-%E6%B7%B1%E5%BA%A6%E5%AD%B8%E7%BF%92-%E5%9F%BA%E7%A4%8E%E4%BB%8B%E7%B4%B9-%E6%90%8D%E5%A4%B1%E5%87%BD%E6%95%B8-loss-function-2dcac5ebb6cb ]

In [ ]:
from sklearn.metrics import accuracy_score, log_loss
pre = clf.predict(x_test_count)
print(accuracy_score(pre, y_test))
prob = clf.predict_proba(x_test_count)
print(log_loss(y_test, prob))

In [ ]:
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(y_test, pre)
pd.DataFrame(mat,
             index=[name + "(原本)" for name in u],
             columns=[name + "(預測)" for name in u])

In [ ]:
p = input("write:")
# 要注意現在只有一個字串,要轉成『一批』字串只有一個內容
test = vec.transform([newscut(p)])
prob = clf.predict_proba(test)
prob_0 = prob[0] #拿整批裡面的唯一一個
print(u)
for i in range(len(u)):
  print(u[i], "的機率:", round(prob_0[i], 3))


簡體轉繁體

opencc-python-reimplemented

s2tw、一般用s2twp
 [ https://pypi.org/project/opencc-python-reimplemented/ ]

！代表指令的意思
!pip install xxxxxxxxx



In [ ]:
# # 安裝簡轉繁的函式庫, 才可以import opencc
# !pip install opencc-python-reimplemented

In [ ]:
from opencc import OpenCC
cc = OpenCC('s2twp')  # convert from Simplified Chinese to Traditional Chinese
# can also set conversion by calling set_conversion
# cc.set_conversion('s2tw')
to_convert = '开放中文转换'
converted = cc.convert(to_convert)
converted